<a href="https://colab.research.google.com/github/rauledop1/TEWS/blob/main/Catastro_BN_chileipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install leafmap rarfile lonboard
#%pip install lonboard

In [3]:
import duckdb
import leafmap
import rarfile
import requests
import os
import pandas as pd
import shutil

In [4]:
#download file function
def download_file(url, output_path):
    # Send a HTTP request to the specified URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Open the output file in binary mode and write the content
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded file to {output_path}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

#uncompress rar file
def extract_rar(rar_file, extract_dir):
    # Open the rar file
    with rarfile.RarFile(rar_file) as rf:
        # Extract all contents to the specified directory
        rf.extractall(extract_dir)
        print(f"Extracted {rar_file} to {extract_dir}")

def extract_table_to_dataframe(url):
    # Send an HTTP request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Use pandas to read the HTML and extract tables
        tables = pd.read_html(response.text)

        # If tables were found, return the first table as a DataFrame
        if tables:
            df = tables[0]
            print(f"Extracted table with {df.shape[0]} rows and {df.shape[1]} columns")
            return df
        else:
            print("No tables found on the webpage.")
            return None
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
        return None

def find_shp_files(base_dir):
    shp_files = []

    # Walk through the directory and subdirectories
    for root, dirs, files in os.walk(base_dir):
        # Search for .shp files
        for file in files:
            if file.endswith('.shp'):
                shp_path = os.path.join(root, file)
                shp_files.append(shp_path)
                print(f"Found shapefile: {shp_path}")

    if shp_files:
        print(f"Found {len(shp_files)} shapefile(s).")
        return shp_files
    else:
        print("No shapefiles found.")
        return None


In [5]:
admin_url='https://raw.githubusercontent.com/rauledop1/TEWS/refs/heads/main/Catastro_BN.csv'

In [6]:
df=pd.read_csv(admin_url)

In [7]:
df

,Category,cod_region,name_region,desc_region,lenaguage,year_update,crs,file_name,file_url
0,region,CL-AI,Aisén del General Carlos Ibañez del Campo,"Aysén, Aisén",es,2011,EPSG:32718,Catastro_RV_R11_2011,https://ide.minagri.gob.cl/geoweb/storage/DESC...
1,region,CL-AN,Antofagasta,NaN,es,2019,EPSG:32719,cbn_reg02_2019,https://ide.minagri.gob.cl/geoweb/storage/DESC...
2,region,CL-AP,Arica y Parinacota,NaN,es,2015,EPSG:32719,catastro_RV_R15_2015,https://ide.minagri.gob.cl/geoweb/storage/DESC...
3,region,CL-AT,Atacama,NaN,es,2018,EPSG:32719,catastro_rv_r03_2018,https://ide.minagri.gob.cl/geoweb/storage/DESC...
4,region,CL-BI,Biobío,NaN,es,2015,EPSG:32718,Catastro_RV_R08_2015,https://ide.minagri.gob.cl/geoweb/storage/DESC...
5,region,CL-CO,Coquimbo,NaN,es,2014,EPSG:32719,Catastro_RV_R04_2014,https://ide.minagri.gob.cl/geoweb/storage/DESC...
6,region,CL-AR,La Araucanía,NaN,es,2017,EPSG:32718,cbn_09reg_2017,https://ide.minagri.gob.cl/geoweb/storage/DESC...
7,region,CL-LI,Libertador General Bernardo O'Higgins,O'Higgins,es,2013,EPSG:32719,Catastro_RV_R06_2013,https://ide.minagri.gob.cl/geoweb/storage/DESC...
8,region,CL-LL,Los Lagos,NaN,es,2018,EPSG:32718,cbn_10reg_2018,https://ide.minagri.gob.cl/geoweb/storage/DESC...
9,region,CL-LR,Los Ríos,NaN,es,2017,EPSG:32718,cbn_14reg_2017,https://ide.minagri.gob.cl/geoweb/storage/DESC...


In [8]:
df=df.query("cod_region == 'CL-MA'")


In [9]:
url_catastro=df.file_url.to_list()[:3]
year_update=df.year_update.to_list()[:3]
cod_region=df.cod_region.to_list()[:3]
crs_file=df.crs.to_list()[:3]

In [10]:
#Create a temporal DB
con = duckdb.connect()

#Load spatial and http extensions
con.sql('''
install httpfs;
load httpfs;
INSTALL spatial;
LOAD spatial;
''')

In [12]:
for i,j,k,c in zip(url_catastro, year_update,cod_region, crs_file):

  compress_file=f'{k}_{j}.rar'
  folder_p=f'{k}_{j}_BN'
  table_name=k.replace('-','_')+f'_{j}'

  download_file(i,compress_file)

  if not os.path.exists(folder_p):
    os.makedirs(folder_p)

  extract_rar(compress_file, folder_p)

  shp=find_shp_files(folder_p)

  shp=shp[0]

  con.sql(f'''
    create table '{table_name}' as
    select * from st_read('{shp}')
    ''')
  con.sql(f'''
      COPY
      (SELECT
        COALESCE(id_uso, NULL) AS id_uso,
        COALESCE(id_subuso, NULL) AS id_subuso,
        COALESCE(id_estruc, NULL) AS id_estruc,
        COALESCE(id_cober, NULL) AS id_cober,
        COALESCE(id_altu, NULL) AS id_altu,
        COALESCE(id_tifo, NULL) AS id_tifo,
        COALESCE(id_stif, NULL) AS id_stif,
        COALESCE(id_esp1, NULL) AS id_esp1,
        COALESCE(id_esp2, NULL) AS id_esp2,
        COALESCE(id_esp3, NULL) AS id_esp3,
        COALESCE(id_esp4, NULL) AS id_esp4,
        COALESCE(id_esp5, NULL) AS id_esp5,
        COALESCE(id_esp6, NULL) AS id_esp6,
        COALESCE(id_esp_cc1, NULL) AS id_esp_cc1,
        COALESCE(id_esp_cc2, NULL) AS id_esp_cc2,
        COALESCE(codreg, NULL) AS codreg,
        COALESCE(codprov, NULL) AS codprov,
        COALESCE(codcom, NULL) AS codcom,
        COALESCE(uso_tierra, NULL) AS uso_tierra,
        COALESCE(uso, NULL) AS uso,
        COALESCE(subuso, NULL) AS subuso,
        COALESCE(estructura, NULL) AS estructura,
        COALESCE(cobertura, NULL) AS cobertura,
        COALESCE(altura, NULL) AS altura,
        COALESCE(tipo_fores, NULL) AS tipo_fores,
        COALESCE(subtipofor, NULL) AS subtipofor,
        COALESCE(especi1_ci, NULL) AS especi1_ci,
        COALESCE(especi2_ci, NULL) AS especi2_ci,
        COALESCE(especi3_ci, NULL) AS especi3_ci,
        COALESCE(especi4_ci, NULL) AS especi4_ci,
        COALESCE(especi5_ci, NULL) AS especi5_ci,
        COALESCE(especi6_ci, NULL) AS especi6_ci,
        COALESCE(especi1_co, NULL) AS especi1_co,
        COALESCE(especi2_co, NULL) AS especi2_co,
        COALESCE(especi3_co, NULL) AS especi3_co,
        COALESCE(especi4_co, NULL) AS especi4_co,
        COALESCE(especi5_co, NULL) AS especi5_co,
        COALESCE(especi6_co, NULL) AS especi6_co,
        COALESCE(esp_c1, NULL) AS esp_c1,
        COALESCE(esp_c2, NULL) AS esp_c2,
        st_Astext(st_transform(geom, '{c}', 'EPSG:4326')) AS geom
      FROM '{table_name}')
      TO
      '/content/drive/MyDrive/Colab Notebooks/catastroBN/'{table_name}'.parquet'
      (FORMAT PARQUET)
      ''')

  os.remove(compress_file)
  shutil.rmtree(folder_p)

Downloaded file to CL-MA_2017.rar
Extracted CL-MA_2017.rar to CL-MA_2017_BN
Found shapefile: CL-MA_2017_BN/cbn_2017_2019_r12/cbn_2017_2019_r12.shp
Found 1 shapefile(s).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

ParserException: Parser Error: syntax error at or near "CL_MA_2017"

In [31]:
con.sql('''
SHOW ALL TABLES
''')

┌──────────┬─────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
con.sql('''
COPY
(SELECT
  COALESCE(id_uso, NULL) AS id_uso,
  COALESCE(id_subuso, NULL) AS id_subuso,
  COALESCE(id_estruc, NULL) AS id_estruc,
  COALESCE(id_cober, NULL) AS id_cober,
  COALESCE(id_altu, NULL) AS id_altu,
  COALESCE(id_tifo, NULL) AS id_tifo,
  COALESCE(id_stif, NULL) AS id_stif,
  COALESCE(id_esp1, NULL) AS id_esp1,
  COALESCE(id_esp2, NULL) AS id_esp2,
  COALESCE(id_esp3, NULL) AS id_esp3,
  COALESCE(id_esp4, NULL) AS id_esp4,
  COALESCE(id_esp5, NULL) AS id_esp5,
  COALESCE(id_esp6, NULL) AS id_esp6,
  COALESCE(id_esp_cc1, NULL) AS id_esp_cc1,
  COALESCE(id_esp_cc2, NULL) AS id_esp_cc2,
  COALESCE(codreg, NULL) AS codreg,
  COALESCE(codprov, NULL) AS codprov,
  COALESCE(codcom, NULL) AS codcom,
  COALESCE(uso_tierra, NULL) AS uso_tierra,
  COALESCE(uso, NULL) AS uso,
  COALESCE(subuso, NULL) AS subuso,
  COALESCE(estructura, NULL) AS estructura,
  COALESCE(cobertura, NULL) AS cobertura,
  COALESCE(altura, NULL) AS altura,
  COALESCE(tipo_fores, NULL) AS tipo_fores,
  COALESCE(subtipofor, NULL) AS subtipofor,
  COALESCE(especi1_ci, NULL) AS especi1_ci,
  COALESCE(especi2_ci, NULL) AS especi2_ci,
  COALESCE(especi3_ci, NULL) AS especi3_ci,
  COALESCE(especi4_ci, NULL) AS especi4_ci,
  COALESCE(especi5_ci, NULL) AS especi5_ci,
  COALESCE(especi6_ci, NULL) AS especi6_ci,
  COALESCE(especi1_co, NULL) AS especi1_co,
  COALESCE(especi2_co, NULL) AS especi2_co,
  COALESCE(especi3_co, NULL) AS especi3_co,
  COALESCE(especi4_co, NULL) AS especi4_co,
  COALESCE(especi5_co, NULL) AS especi5_co,
  COALESCE(especi6_co, NULL) AS especi6_co,
  COALESCE(esp_c1, NULL) AS esp_c1,
  COALESCE(esp_c2, NULL) AS esp_c2,
  st_Astext(st_transform(geom, 'EPSG:32718', 'EPSG:4326')) AS geom
FROM CL_MA_2017)
TO
'/content/drive/MyDrive/Colab Notebooks/catastroBN/CL_MA_2017.parquet'
(FORMAT PARQUET)
''')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))